In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from matplotlib.pylab import plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
#ACS 2012-2016 data comes from 4 separate data sets that have to be combined: demo/econ/househould/socio

In [2]:
#loading demographic data 
demo = pd.read_csv('demo_2016acs5yr_nta.csv') 
demo.head()

,GeoType,GeogName,GeoID,Borough,Pop_1E,Pop_1M,Pop_1C,Pop_1P,Pop_1Z,MaleE,...,AsnOAsnE,AsnOAsnM,AsnOAsnC,AsnOAsnP,AsnOAsnZ,Asn2plE,Asn2plM,Asn2plC,Asn2plP,Asn2plZ
0,NTA2010,Bath Beach,BK27,Brooklyn,"33,163","1,246",2.3,100.0,NaN,"16,686",...,47.0,42.0,54.9,0.4,0.0,152.0,103.0,41.4,1.3,1.0
1,NTA2010,Bay Ridge,BK31,Brooklyn,"79,134","1,724",1.3,100.0,NaN,"37,829",...,113.0,113.0,60.9,1.0,1.0,93.0,61.0,40.2,0.8,1.0
2,NTA2010,Bedford,BK75,Brooklyn,"75,318","1,993",1.6,100.0,NaN,"36,491",...,26.0,29.0,67.8,1.1,1.0,56.0,52.0,56.7,2.4,2.0
3,NTA2010,Bensonhurst East,BK29,Brooklyn,"64,267","1,675",1.6,100.0,NaN,"31,283",...,47.0,57.0,73.2,0.2,0.0,232.0,175.0,45.9,0.9,1.0
4,NTA2010,Bensonhurst West,BK28,Brooklyn,"91,646","1,975",1.3,100.0,NaN,"44,651",...,279.0,190.0,41.5,0.7,1.0,428.0,202.0,28.7,1.1,1.0


In [3]:
demo.head()

,GeoType,GeogName,GeoID,Borough,Pop_1E,Pop_1M,Pop_1C,Pop_1P,Pop_1Z,MaleE,...,AsnOAsnE,AsnOAsnM,AsnOAsnC,AsnOAsnP,AsnOAsnZ,Asn2plE,Asn2plM,Asn2plC,Asn2plP,Asn2plZ
0,NTA2010,Bath Beach,BK27,Brooklyn,"33,163","1,246",2.3,100.0,NaN,"16,686",...,47.0,42.0,54.9,0.4,0.0,152.0,103.0,41.4,1.3,1.0
1,NTA2010,Bay Ridge,BK31,Brooklyn,"79,134","1,724",1.3,100.0,NaN,"37,829",...,113.0,113.0,60.9,1.0,1.0,93.0,61.0,40.2,0.8,1.0
2,NTA2010,Bedford,BK75,Brooklyn,"75,318","1,993",1.6,100.0,NaN,"36,491",...,26.0,29.0,67.8,1.1,1.0,56.0,52.0,56.7,2.4,2.0
3,NTA2010,Bensonhurst East,BK29,Brooklyn,"64,267","1,675",1.6,100.0,NaN,"31,283",...,47.0,57.0,73.2,0.2,0.0,232.0,175.0,45.9,0.9,1.0
4,NTA2010,Bensonhurst West,BK28,Brooklyn,"91,646","1,975",1.3,100.0,NaN,"44,651",...,279.0,190.0,41.5,0.7,1.0,428.0,202.0,28.7,1.1,1.0


In [4]:
#Creating dataframe with selected features: popultion, pop male, pop female, median age, hispanic, nonhispanic, etc

demo1 = demo[['GeoType', 'GeogName', 'GeoID', 'Borough','Pop_1E','MaleE','FemE',
              'PopU181E','Pop65pl1E','MdAgeE','Hsp1E','NHspE','WtNHE','BlNHE',
              'AIANNHE','AsnNHE','NHPINHE','OthNHE','Rc2plNHE']].copy()


In [5]:
demo1.head()

,GeoType,GeogName,GeoID,Borough,Pop_1E,MaleE,FemE,PopU181E,Pop65pl1E,MdAgeE,Hsp1E,NHspE,WtNHE,BlNHE,AIANNHE,AsnNHE,NHPINHE,OthNHE,Rc2plNHE
0,NTA2010,Bath Beach,BK27,Brooklyn,"33,163","16,686","16,477","7,103","4,888",38.3,"5,861","27,302","14,490",404,11.0,"11,975",29.0,71,322
1,NTA2010,Bay Ridge,BK31,Brooklyn,"79,134","37,829","41,305","15,110","12,380",39.5,"14,134","65,000","50,197","1,511",181.0,"11,515",19.0,151,"1,426"
2,NTA2010,Bedford,BK75,Brooklyn,"75,318","36,491","38,827","22,379","5,816",29.1,"14,682","60,636","26,407","30,566",66.0,"2,290",0.0,149,"1,158"
3,NTA2010,Bensonhurst East,BK29,Brooklyn,"64,267","31,283","32,984","12,773","10,107",39.6,"7,923","56,344","29,894",531,0.0,"24,858",0.0,348,713
4,NTA2010,Bensonhurst West,BK28,Brooklyn,"91,646","44,651","46,995","19,288","13,071",37.8,"13,378","78,268","39,112",801,111.0,"37,132",0.0,319,793


In [6]:
#economic indicators 
econ = pd.read_csv('econ_2016acs5yr_nta.csv') 
econ.head()

,GeoType,GeogName,GeoID,Borough,Pop16plE,Pop16plM,Pop16plC,Pop16plP,Pop16plZ,LFE,...,Pv400t499E,Pv400t499M,Pv400t499C,Pv400t499P,Pv400t499Z,Pv500plE,Pv500plM,Pv500plC,Pv500plP,Pv500plZ
0,NTA2010,Bath Beach,BK27,Brooklyn,"26,855",943,2.1,100.0,NaN,"16,006",...,"3,464",434,7.6,10.5,1.0,"8,268",599,4.4,25.1,2.0
1,NTA2010,Bay Ridge,BK31,Brooklyn,"65,479","1,260",1.2,100.0,NaN,"42,176",...,"7,564",572,4.6,9.6,1.0,"28,491","1,041",2.2,36.1,1.0
2,NTA2010,Bedford,BK75,Brooklyn,"54,904","1,390",1.5,100.0,NaN,"36,993",...,"5,175",561,6.6,6.9,1.0,"12,144",846,4.2,16.2,1.0
3,NTA2010,Bensonhurst East,BK29,Brooklyn,"52,857","1,307",1.5,100.0,NaN,"32,004",...,"5,634",522,5.6,8.8,1.0,"12,868",812,3.8,20.1,1.0
4,NTA2010,Bensonhurst West,BK28,Brooklyn,"74,470","1,568",1.3,100.0,NaN,"45,842",...,"7,969",641,4.9,8.7,1.0,"17,583",934,3.2,19.2,1.0


In [7]:
econ1 = econ[['GeogName', 'HHIU10E','HHI10t14E','HHI15t24E','HHI25t34E',
              'HHI35t49E','HHI50t74E','HHI75t99E','HI100t149E','HI150t199E',
              'HHI200plE','MdHHIncE','MnHHIncE']].copy()

#NOTE: income data is according to household; selected household income data according to different brackets 

In [8]:
econ1.head()

,GeogName,HHIU10E,HHI10t14E,HHI15t24E,HHI25t34E,HHI35t49E,HHI50t74E,HHI75t99E,HI100t149E,HI150t199E,HHI200plE,MdHHIncE,MnHHIncE
0,Bath Beach,895,618,"1,223",927,"1,290","2,074","1,352","1,758",706,585,"58,243","75,950"
1,Bay Ridge,"2,313","1,573","3,235","3,047","3,542","5,570","4,108","4,827","2,497","2,775","63,539","86,319"
2,Bedford,"3,673","2,317","3,385","2,509","3,124","3,706","2,269","2,847","1,195","1,034","39,970","62,616"
3,Bensonhurst East,"2,356","1,413","2,718","2,185","2,655","3,898","2,558","2,416","1,144",844,"48,097","65,715"
4,Bensonhurst West,"2,330","1,746","3,383","3,246","4,036","5,117","3,428","3,743","1,649","1,231","51,035","70,198"


In [9]:
#did not select any data here - very detailed household info
hous = pd.read_csv('hous_2016acs5yr_nta.csv') 
hous.head()

,GeoType,GeogName,GeoID,Borough,HU1E,HU1M,HU1C,HU1P,HU1Z,OcHU1E,...,GRPI50plE,GRPI50plM,GRPI50plC,GRPI50plP,GRPI50plZ,GRPINtCE,GRPINtCM,GRPINtCC,GRPINtCP,GRPINtCZ
0,NTA2010,Bath Beach,BK27,Brooklyn,"12,090",156.0,0.8,100.0,NaN,"11,428",...,"1,975",244.0,7.5,30.0,4.0,379,104.0,16.7,NaN,NaN
1,NTA2010,Bay Ridge,BK31,Brooklyn,"37,242",295.0,0.5,100.0,NaN,"33,487",...,"5,640",427.0,4.6,27.8,2.0,"1,165",224.0,11.7,NaN,NaN
2,NTA2010,Bedford,BK75,Brooklyn,"29,106",251.0,0.5,100.0,NaN,"26,059",...,"6,226",458.0,4.5,30.9,2.0,"1,070",216.0,12.3,NaN,NaN
3,NTA2010,Bensonhurst East,BK29,Brooklyn,"23,851",240.0,0.6,100.0,NaN,"22,187",...,"4,400",385.0,5.3,32.6,3.0,"1,055",200.0,11.5,NaN,NaN
4,NTA2010,Bensonhurst West,BK28,Brooklyn,"31,701",284.0,0.5,100.0,NaN,"29,909",...,"5,862",435.0,4.5,31.6,2.0,"1,039",192.0,11.2,NaN,NaN


In [9]:
#socio/edu indicators
soc = pd.read_csv('soc_2016acs5yr_nta.csv') 
soc.head()

,GeoType,GeogName,GeoID,Borough,HH1E,HH1M,HH1C,HH1P,HH1Z,Fam1E,...,OthrE,OthrM,OthrC,OthrP,OthrZ,UnclsNRE,UnclsNRM,UnclsNRC,UnclsNRP,UnclsNRZ
0,NTA2010,Bath Beach,BK27,Brooklyn,"11,428",209,1.1,100.0,NaN,"8,207",...,"15,051","1,186",4.8,45.4,3.0,"6,697","1,100",10.0,20.2,3.0
1,NTA2010,Bay Ridge,BK31,Brooklyn,"33,487",442,0.8,100.0,NaN,"18,935",...,"27,814","1,530",3.3,35.1,2.0,"7,423",806,6.6,9.4,1.0
2,NTA2010,Bedford,BK75,Brooklyn,"26,059",401,0.9,100.0,NaN,"14,540",...,"41,891","1,898",2.8,55.6,2.0,"9,693","1,035",6.5,12.9,1.0
3,NTA2010,Bensonhurst East,BK29,Brooklyn,"22,187",338,0.9,100.0,NaN,"16,276",...,"34,121","1,732",3.1,53.1,2.0,"9,077","1,007",6.7,14.1,2.0
4,NTA2010,Bensonhurst West,BK28,Brooklyn,"29,909",364,0.7,100.0,NaN,"21,832",...,"47,340","1,974",2.5,51.7,2.0,"15,409","1,377",5.4,16.8,2.0


In [10]:
soc1 = soc[['GeogName', 'EA_P25plE','EA_LT9GE','EA_9t12NDE',
            'EA_HScGrdE','EA_SClgNDE','EA_AscDE','EA_BchDE','EA_GrdPfDE',
            'EA_LTHSGrE','EA_BchDHE']].copy()


#NOTE: selected educational attainment (persons)

In [11]:
soc1.head()

,GeogName,EA_P25plE,EA_LT9GE,EA_9t12NDE,EA_HScGrdE,EA_SClgNDE,EA_AscDE,EA_BchDE,EA_GrdPfDE,EA_LTHSGrE,EA_BchDHE
0,Bath Beach,"23,822","3,146","2,326","6,188","2,486","1,667","5,345","2,664","5,472","8,009"
1,Bay Ridge,"58,178","4,106","3,900","11,165","8,251","3,746","15,823","11,187","8,006","27,010"
2,Bedford,"44,766","3,908","5,791","11,608","6,838","2,215","9,148","5,258","9,699","14,406"
3,Bensonhurst East,"46,010","6,856","4,700","12,871","4,433","3,262","9,412","4,476","11,556","13,888"
4,Bensonhurst West,"65,016","11,095","7,191","18,127","6,748","4,796","12,275","4,784","18,286","17,059"


In [12]:
#loaded gardens data 
gardens = pd.read_csv('NYC_Community_Gardens_combinedv1.csv') 
gardens.head()


,Unnamed: 0,Address,Latitude,Longitude,NTA
0,0,735 East 211th St,40.877499,-73.863489,Williamsbridge-Olinville
1,1,1818 Bathgate Ave,40.845051,-73.897747,Claremont-Bathgate
2,2,1017 Teller Ave,40.828150,-73.914356,East Concourse-Concourse Village
3,3,1328 Clay Ave,40.834934,-73.908892,East Concourse-Concourse Village
4,4,2044 Prospect Ave,40.845818,-73.887851,East Tremont


In [13]:
gardens.shape

(575, 5)

In [14]:
#merging diff ACS datasets: demographic/econ/socio indicators

ACSmerge = pd.merge(pd.merge(demo1,econ1,on='GeogName'),soc1,on='GeogName')

In [15]:
ACSmerge.head()

,GeoType,GeogName,GeoID,Borough,Pop_1E,MaleE,FemE,PopU181E,Pop65pl1E,MdAgeE,...,EA_P25plE,EA_LT9GE,EA_9t12NDE,EA_HScGrdE,EA_SClgNDE,EA_AscDE,EA_BchDE,EA_GrdPfDE,EA_LTHSGrE,EA_BchDHE
0,NTA2010,Bath Beach,BK27,Brooklyn,"33,163","16,686","16,477","7,103","4,888",38.3,...,"23,822","3,146","2,326","6,188","2,486","1,667","5,345","2,664","5,472","8,009"
1,NTA2010,Bay Ridge,BK31,Brooklyn,"79,134","37,829","41,305","15,110","12,380",39.5,...,"58,178","4,106","3,900","11,165","8,251","3,746","15,823","11,187","8,006","27,010"
2,NTA2010,Bedford,BK75,Brooklyn,"75,318","36,491","38,827","22,379","5,816",29.1,...,"44,766","3,908","5,791","11,608","6,838","2,215","9,148","5,258","9,699","14,406"
3,NTA2010,Bensonhurst East,BK29,Brooklyn,"64,267","31,283","32,984","12,773","10,107",39.6,...,"46,010","6,856","4,700","12,871","4,433","3,262","9,412","4,476","11,556","13,888"
4,NTA2010,Bensonhurst West,BK28,Brooklyn,"91,646","44,651","46,995","19,288","13,071",37.8,...,"65,016","11,095","7,191","18,127","6,748","4,796","12,275","4,784","18,286","17,059"


In [20]:
#Dropped NANs, result is 195 NTAs

ACSmerge = ACSmerge.dropna(how='all')
ACSmerge.shape

(195, 41)

In [16]:
ACSmerge.columns

Index(['GeoType', 'GeogName', 'GeoID', 'Borough', 'Pop_1E', 'MaleE', 'FemE',
       'PopU181E', 'Pop65pl1E', 'MdAgeE', 'Hsp1E', 'NHspE', 'WtNHE', 'BlNHE',
       'AIANNHE', 'AsnNHE', 'NHPINHE', 'OthNHE', 'Rc2plNHE', 'HHIU10E',
       'HHI10t14E', 'HHI15t24E', 'HHI25t34E', 'HHI35t49E', 'HHI50t74E',
       'HHI75t99E', 'HI100t149E', 'HI150t199E', 'HHI200plE', 'MdHHIncE',
       'MnHHIncE', 'EA_P25plE', 'EA_LT9GE', 'EA_9t12NDE', 'EA_HScGrdE',
       'EA_SClgNDE', 'EA_AscDE', 'EA_BchDE', 'EA_GrdPfDE', 'EA_LTHSGrE',
       'EA_BchDHE'],
      dtype='object')

In [17]:
gardens.rename(columns = {'NTA': 'GeogName'}, inplace = True) 

In [18]:
gardens = gardens[['Address','Latitude','Longitude','GeogName']].copy()
gardens.head()

,Address,Latitude,Longitude,GeogName
0,735 East 211th St,40.877499,-73.863489,Williamsbridge-Olinville
1,1818 Bathgate Ave,40.845051,-73.897747,Claremont-Bathgate
2,1017 Teller Ave,40.828150,-73.914356,East Concourse-Concourse Village
3,1328 Clay Ave,40.834934,-73.908892,East Concourse-Concourse Village
4,2044 Prospect Ave,40.845818,-73.887851,East Tremont


In [31]:
# Count of gardens by NTA 
gardens['GeogName'].value_counts()

East New York                                            36
Lower East Side                                          26
East Harlem North                                        24
Bedford                                                  23
Stuyvesant Heights                                       20
                                                         ..
Bensonhurst East                                          1
Old Astoria                                               1
Kingsbridge Heights                                       1
Spuyten Duyvil-Kingsbridge                                1
Breezy Point-Belle Harbor-Rockaway Park-Broad Channel     1
Name: GeogName, Length: 95, dtype: int64

In [32]:
#95 NTAs with gardens 
gardens['GeogName'].nunique()

95

In [19]:
gardens.isnull().sum()

Address       0
Latitude      2
Longitude     2
GeogName     50
dtype: int64

In [21]:
gardens_unique = gardens.groupby('GeogName').count()

In [22]:
gardens_unique

,Address,Latitude,Longitude
GeogName,,,
Allerton-Pelham Gardens,1,1,1
Astoria,1,1,1
Baisley Park,3,3,3
Battery Park City-Lower Manhattan,1,1,1
Bedford,23,23,23
...,...,...,...
Williamsbridge-Olinville,1,1,1
Williamsburg,1,1,1
Windsor Terrace,1,1,1


In [26]:
gardens['Address'].nunique()

575